# Introduction

Using this Python notebook you will:

1.  Understand three Chicago datasets  
2.  Load the three datasets into three tables in a Db2 database
3.  Execute SQL queries to answer some questions 


### Connect to the database

Let us first load the SQL extension and establish a connection with the database



In [ ]:
# !pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
# !pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
# !pip install ipython-sql

In [1]:
%load_ext sql

In [2]:
# Remember the connection string is of the format:
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
# Enter the connection string for your Db2 on Cloud database instance below
%sql ibm_db_sa://tcz34963:3SyPsHL0dpSx6N88@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB?security=SSL

'Connected: tcz34963@BLUDB'

## Questions

Now lets write and execute SQL queries to answer some Questions

### Question 1

##### Find the total number of crimes recorded in the CRIME table.


In [6]:
%%sql
SELECT COUNT(id) FROM CHICAGO_CRIME_DATA

 * ibm_db_sa://tcz34963:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB
Done.


1
533


### Question 2

##### List community areas with per capita income less than 11000.


In [7]:
%%sql
SELECT COMMUNITY_AREA_NUMBER , COMMUNITY_AREA_NAME FROM CENSUS_DATA
WHERE PER_CAPITA_INCOME < 11000

 * ibm_db_sa://tcz34963:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB
Done.


community_area_number,community_area_name
26,West Garfield Park
30,South Lawndale
37,Fuller Park
54,Riverdale


### Question 3

##### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)



In [22]:
%%sql
SELECT id,CASE_NUMBER , DESCRIPTION FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE  '%MINOR%'

 * ibm_db_sa://tcz34963:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB
Done.


id,case_number,description
3987219,HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
3266814,HK238408,ILLEGAL CONSUMPTION BY MINOR


### Question 4

##### List all kidnapping crimes involving a child?


In [21]:
%%sql
SELECT id,CASE_NUMBER , DESCRIPTION FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE  '%CHILD%' AND 
PRIMARY_TYPE = 'KIDNAPPING'

 * ibm_db_sa://tcz34963:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB
Done.


id,case_number,description
5276766,HN144152,CHILD ABDUCTION/STRANGER


### Question 5

##### What kinds of crimes were recorded at schools?


In [43]:
%%sql
SELECT Distinct(PRIMARY_TYPE) FROM CHICAGO_CRIME_DATA
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * ibm_db_sa://tcz34963:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB
Done.


primary_type
ASSAULT
BATTERY
CRIMINAL DAMAGE
CRIMINAL TRESPASS
NARCOTICS
PUBLIC PEACE VIOLATION


### Question 6

##### List the average safety score for each type of school.


In [37]:
%%sql
SELECT "Elementary, Middle, or High School" ,AVG(SAFETY_SCORE) as average_safety_score FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School"

 * ibm_db_sa://tcz34963:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB
Done.


"Elementary, Middle, or High School",average_safety_score
ES,49
HS,49
MS,48


### Question 7

##### List 5 community areas with highest % of households below poverty line 


In [40]:
%%sql 
SELECT  COMMUNITY_AREA_NUMBER , COMMUNITY_AREA_NAME ,PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5;

 * ibm_db_sa://tcz34963:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB
Done.


community_area_number,community_area_name,percent_households_below_poverty
54,Riverdale,56.5
37,Fuller Park,51.2
68,Englewood,46.6
29,North Lawndale,43.1
27,East Garfield Park,42.4


### Question 8

##### Which community area is most crime prone?


In [50]:
%%sql
SELECT COMMUNITY_AREA_NUMBER , count(id) as number_of_crime FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY number_of_crime DESC
LIMIT 1


 * ibm_db_sa://tcz34963:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB
Done.


community_area_number,number_of_crime
25,43


Double-click **here** for a hint

<!--
Query for the 'community area number' that is most crime prone.
-->


### Question 9

##### Use a sub-query to find the name of the community area with highest hardship index


In [41]:
%%sql
SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA 
WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA )

 * ibm_db_sa://tcz34963:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB
Done.


community_area_name
Riverdale


### Question 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?


In [37]:
%%sql
SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (SELECT MAX(countt)
FROM (SELECT count(id) as countt FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER ));

 * ibm_db_sa://tcz34963:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/BLUDB
Done.


community_area_name
South Shore
